In [1]:
import sys

sys.path.append("../")

import torch
import torch.utils as utils

import pytorch_lightning as pl

from torchemlp.groups import SO, O, S, Z
from torchemlp.nn.equivariant import EMLP
from torchemlp.nn.runners import (
    FuncMSERegressionLightning,
    DynamicsL2RegressionLightning,
)
from torchemlp.nn.utils import Standardize
from torchemlp.datasets import DoublePendulum

# torch.set_default_dtype(torch.float32)

In [2]:
TRAINING_SET_SIZE = 5_000
VALIDATION_SET_SIZE = 1_000
TEST_SET_SIZE = 1_000

DT = 0.1
T = 30.0

BATCH_SIZE = 500

N_EPOCHS = 5  # min(int(900_000 / TRAINING_SET_SIZE), 1000)
# N_EPOCHS = 100  # min(int(900_000 / TRAINING_SET_SIZE), 1000)

N_CHANNELS = 384
N_LAYERS = 3

DL_WORKERS = 0

In [3]:
dataset = DoublePendulum(
    TRAINING_SET_SIZE + VALIDATION_SET_SIZE + TEST_SET_SIZE,
    DT,
    T,
)
f"Input type: {dataset.repin(dataset.G)}, output type: {dataset.repout(dataset.G)}"

'Input type: 4V⁰+4V, output type: 8V⁰'

In [4]:
dataset.X[0].shape

torch.Size([7000, 12])

In [5]:
split_data = utils.data.random_split(
    dataset, [TRAINING_SET_SIZE, VALIDATION_SET_SIZE, TEST_SET_SIZE]
)

train_loader = utils.data.DataLoader(
    split_data[0], batch_size=BATCH_SIZE, num_workers=DL_WORKERS, shuffle=True
)
val_loader = utils.data.DataLoader(
    split_data[1], batch_size=BATCH_SIZE, num_workers=DL_WORKERS
)
test_loader = utils.data.DataLoader(
    split_data[2], batch_size=BATCH_SIZE, num_workers=DL_WORKERS
)

In [6]:
model = Standardize(
    EMLP(dataset.repin, dataset.repout, dataset.G, N_CHANNELS, N_LAYERS), dataset.stats
)  # .cuda()
plmodel = DynamicsL2RegressionLightning(model)
plmodel.to("mps")

DynamicsL2RegressionLightning(
  (model): Standardize(
    (model): EMLP(
      (network): Sequential(
        (0): EMLPBlock(
          (linear): EquivariantLinear()
          (bilinear): EquivariantBiLinear()
          (nonlinear): GatedNonlinearity()
        )
        (1): EMLPBlock(
          (linear): EquivariantLinear()
          (bilinear): EquivariantBiLinear()
          (nonlinear): GatedNonlinearity()
        )
        (2): EMLPBlock(
          (linear): EquivariantLinear()
          (bilinear): EquivariantBiLinear()
          (nonlinear): GatedNonlinearity()
        )
        (3): EquivariantLinear()
      )
    )
  )
)

In [7]:
trainer = pl.Trainer(
    limit_train_batches=BATCH_SIZE, max_epochs=N_EPOCHS  # , accelerator="gpu"
)
trainer.fit(plmodel, train_loader, val_loader)
trainer.test(plmodel, test_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/rytse/mambaforge/envs/l2e_mac/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(

  | Name  | Type        | Params
--------------------------------------
0 | model | Standardize | 570 K 
--------------------------------------
570 K     Trainable params
0         Non-trainable params
570 K     Total params
2.280     Total estimated model params

Sanity Checking DataLoader 0:   0%|                                                                                                                                                               | 0/2 [00:00<?, ?it/s]

/Users/rytse/mambaforge/envs/l2e_mac/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


RuntimeError: Tensor for argument #1 'mat1' is on CPU, but expected it to be on GPU (while checking arguments for mm)